Copyright (c) 2025 Ken Barker

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Compare GeographicLib Inverse Distances and Azimuths

Compare the distances and azimuths calculated by the GeographicLib `Geodesic::Inverse` method at standard (double) precision using both g++ and clang compilers with the reference values from Charles Karney's [Test data for geodesics](https://geographiclib.sourceforge.io/C++/doc/geodesic.html#testgeod).

The values are produced by the via-ellipsoid_test program built with `-DCPP_ACCURACY_TEST=ON` and `-DUSE_GEOGRAPHICLIB=ON` with the `OUTPUT_GEOGRAPHICLIB_VALUES` macro set.
The results are piped into a file.
Note: the first line of the file must be removed before it is copied into the `GEODTEST_DIR` directory.

In [1]:
import numpy as np
import polars as pl
from scipy import stats
from enum import Enum
from os import environ, path

# The colums of the GeodTest.dat file
class Column(Enum):
    latitude_1   = 0
    longitude_1  = 1
    azimuth_1    = 2
    latitude_2   = 3
    longitude_2  = 4
    azimuth_2    = 5
    distance_m   = 6
    distance_deg = 7
    m12          = 8
    area         = 9

# The location of the files
file_path = environ.get('GEODTEST_DIR')

## Read the GeodTest.dat file into a polars LazyFrame

In [2]:
# Read the geodesic test data file into a polars LazyFrame: lf
# Select all the data entries
size = 500000
pathname = path.join(file_path, 'GeodTest.dat.gz')
ref_lf = pl.scan_csv(pathname, separator=' ', has_header=False).select(
    ['column_1', 'column_2', 'column_3', 'column_4', 'column_5', 'column_6', 'column_7']
).collect()
ref_lf.schema

Schema([('column_1', Float64),
        ('column_2', Int64),
        ('column_3', Float64),
        ('column_4', Float64),
        ('column_5', Float64),
        ('column_6', Float64),
        ('column_7', Float64)])

## Read the GeographicLib gcc output file into a polars LazyFrame

In [3]:
pathname = path.join(file_path, 'geographiclib_data_gcc_v15_1_1.dat.gz')
gcc_lf = pl.scan_csv(pathname, separator=' ', has_header=False).select(
    ['column_1', 'column_2', 'column_3', 'column_4', 'column_5', 'column_6', 'column_7']
).collect()
gcc_lf.schema

Schema([('column_1', Float64),
        ('column_2', Int64),
        ('column_3', Float64),
        ('column_4', Float64),
        ('column_5', Float64),
        ('column_6', Float64),
        ('column_7', Float64)])

## Read the GeographicLib clang output file into a polars LazyFrame

In [4]:
pathname = path.join(file_path, 'geographiclib_data_clang_v20_1_3.dat.gz')
clang_lf = pl.scan_csv(pathname, separator=' ', has_header=False).select(
    ['column_1', 'column_2', 'column_3', 'column_4', 'column_5', 'column_6', 'column_7']
).collect()
clang_lf.schema

Schema([('column_1', Float64),
        ('column_2', Int64),
        ('column_3', Float64),
        ('column_4', Float64),
        ('column_5', Float64),
        ('column_6', Float64),
        ('column_7', Float64)])

In [5]:
ref_lf.equals(gcc_lf)

False

In [6]:
ref_lf.equals(clang_lf)

False

In [7]:
gcc_lf.equals(clang_lf)

True

## The gcc and clang outputs are identical

So just compare gcc compiler output with reference data

### Distance

In [8]:
delta_ref_gcc_s12 = np.subtract(gcc_lf[Column.distance_m.value], ref_lf[Column.distance_m.value])

In [9]:
delta_ref_gcc_s12.max()

0.0

In [10]:
delta_ref_gcc_s12.min()

-9.313225746154785e-10

### Azimuths

In [11]:
delta_ref_gcc_azi1 = np.abs(np.subtract(gcc_lf[Column.azimuth_1.value], ref_lf[Column.azimuth_1.value]))
delta_ref_gcc_azi1.max()

1.4210854715202004e-14

In [12]:
delta_ref_gcc_azi2 = np.abs(np.subtract(gcc_lf[Column.azimuth_2.value], ref_lf[Column.azimuth_2.value]))
delta_ref_gcc_azi2.max()

1.862645149230957e-09